## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
1,2025-07-09-01-00-00 +0000,wsj,Weeks after Elon Musk left his role at DOGE am...,https://www.wsj.com/politics/policy/doge-elon-...
2,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
3,2025-07-09-00-54-29 +0000,nyt,U.S. Botched a Deal to Swap Venezuelans in El ...,https://www.nytimes.com/2025/07/08/world/ameri...
4,2025-07-09-00-51-12 +0000,nyt,Trump’s Frustration With Putin Preceded Resump...,https://www.nytimes.com/2025/07/08/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,18
0,president,6
8,tariffs,6
66,texas,4
211,tariff,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
52,2025-07-08-03-54-00 +0000,wsj,President Trump said the U.S. would resume pro...,https://www.wsj.com/politics/national-security...,49
1,2025-07-09-01-00-00 +0000,wsj,Weeks after Elon Musk left his role at DOGE am...,https://www.wsj.com/politics/policy/doge-elon-...,47
0,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,47
43,2025-07-08-11-00-00 +0000,wsj,A group tied to House Republican leadership is...,https://www.wsj.com/politics/policy/republican...,47
19,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,44


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
52,49,2025-07-08-03-54-00 +0000,wsj,President Trump said the U.S. would resume pro...,https://www.wsj.com/politics/national-security...
19,26,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
1,23,2025-07-09-01-00-00 +0000,wsj,Weeks after Elon Musk left his role at DOGE am...,https://www.wsj.com/politics/policy/doge-elon-...
2,23,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
43,23,2025-07-08-11-00-00 +0000,wsj,A group tied to House Republican leadership is...,https://www.wsj.com/politics/policy/republican...
8,18,2025-07-09-00-00-00 +0000,wsj,MAGA influencers are calling for Attorney Gene...,https://www.wsj.com/politics/policy/pam-bondi-...
0,17,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
23,16,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
9,13,2025-07-08-23-44-53 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo
12,13,2025-07-08-23-01-00 +0000,wsj,Donald Trump approaches the interior design of...,https://www.wsj.com/politics/trump-cabinet-mee...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
